In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd

# MongoDB Example - for Accountant reinsurance loss data analysis 

<!-- PELICAN_BEGIN_SUMMARY -->

MongoDB is a document-oriented database. Instead of storing your data in tables made out of individual rows,
like a relational database does, it stores your data in collections made out of individual documents.
In MongoDB, a document is a big JSON blob with no particular format or schema.

You can have all your data in one single table or collection.

<!-- PELICAN_END_SUMMARY -->

### MongoDB for Reinsurance Data Analysis
- Create sample data in MongoDB
  - reinsurance treaty by reinsurer/year/treaty type
- Read webpage data into MongoDB and 
  - example : read twitter page about what type of insurance people are interesting buying
- Retrieve from MongoDB to Excel for data analysis
- Read investment gain and loss from broker website into MongoDB for analysis
- how to use BSON-NumPy to pull data out of Mongo and into pandas

### *Making a Connection with MongoClient*
-  Below link is an introduction tutorial to working with MongoDB and PyMongo
-  ref: http://api.mongodb.com/python/current/tutorial.html

In [2]:
# Making a Connection with MongoClient
# Import pymongo
import pymongo

# The first step when working with PyMongo is to create a MongoClient to the running mongod instance
from pymongo import MongoClient

# Connect on the default host and port.
client = MongoClient()

# We can also specify the host and port explicitly
client = MongoClient('localhost', 27017)

#### *Getting a Database*
- A single instance of MongoDB can support multiple independent databases.
- When working with PyMongo you access databases using attribute style access on MongoClient instances.
- database name can not use attribute style access (like sample-data), but sample_data is okay

In [3]:
db = client.test_database

collection = db.test_collection

# cant use attribute style
# you can use dictionary style access instead
db = client['test-database']

collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database'), 'test_collection')

#### *Document*
- Data in MongoDB is represented (and stored) using JSON-style documents. 
- In PyMongo we use dictionaries to represent documents.
- As an example, the following dictionary might be used to represent a reinsurance treaty type by reinsurer and year:
- Documents can contain native Python types (like datetime.datetime instances) which will be automatically converted to and from the appropriate BSON types.

In [4]:
import datetime
post = {"reinsurer": "AIG",
        "treaty": "XOL layer",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

#### *Inserting a Document*
- When a document is inserted a special key, "_id", is automatically added if the document doesn’t already contain an "_id" key.
- The value of "_id" must be unique across the collection. insert_one() returns an instance of InsertOneResult. 

In [5]:
# To insert a document into a collection we can use the insert_one() method:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5b747db3760df74924ecbe28')

#### * After inserting the first document, the posts collection has actually been created on the server.*
- We can verify this by listing all of the collections in our database:

In [6]:
db.collection_names(include_system_collections=False)

C:\apps\python36\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['posts', 'profiles']

In [7]:
post2 = {"reinsurer": "Swiss Re",
        "treaty": "Clash Layer",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

post_id = posts.insert_one(post2).inserted_id
post_id

ObjectId('5b747db8760df74924ecbe29')

#### * Getting a Single Document With find_one()*
- The most basic type of query that can be performed in MongoDB is find_one().
- This method returns a single document matching a query (or None if there are no matches).
- It is useful when you know there is only one matching document, or
- are only interested in the first match. Here we use find_one() to get the first document from the posts collection:

In [8]:
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}


In [9]:
pprint.pprint(posts.find_one({"reinsurer": "AIG"}))

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}


In [10]:
pprint.pprint(posts.find_one({"reinsurer": "ACE"}))

None


#### * Querying By ObjectId*
- We can also find a post by its _id, which in our example is an ObjectId:

In [11]:
post_id  ##output is an object

ObjectId('5b747db8760df74924ecbe29')

In [12]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('5b747db8760df74924ecbe29'),
 'date': datetime.datetime(2018, 8, 15, 19, 23, 36, 998000),
 'reinsurer': 'Swiss Re',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'Clash Layer'}


#### * Note that an ObjectId is not the same as its string representation:*

In [13]:
post_id_as_str = str(post_id)
post_id_as_str  ## output is a string

'5b747db8760df74924ecbe29'

In [14]:
posts.find_one({"_id": post_id_as_str}) # No result

#### *Get URL data*
- A common task in web applications is to get an ObjectId from the request URL and find the matching document.
- It’s necessary in this case to convert the ObjectId from a string before passing it to find_one:

In [15]:
# follow up @8/14/18
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

#### * A Note On Unicode Strings*
- You probably noticed that the regular Python strings we stored earlier look different when retrieved from the server (e.g. u’AIG’ instead of ‘AIG’).
- A short explanation is in order.
- MongoDB stores data in BSON format. BSON strings are UTF-8 encoded so PyMongo must ensure that any strings it stores contain only valid UTF-8 data.
- Regular strings (<type ‘str’>) are validated and stored unaltered. Unicode strings (<type ‘unicode’>) are encoded UTF-8 first. 
- The reason our example string is represented in the Python shell as u’AIG’ instead of ‘AIG’ 
- is that PyMongo decodes each BSON string to a Python unicode string, not a regular str.

#### * Bulk Inserts*
- we can also perform bulk insert operations, by passing a list as the first argument to insert_many(). 
- This will insert each document in the list, sending only a single command to the server:
- different shape, but can handle with new title,can add to the collection... because its not a table

In [16]:
# new_posts[1] has a different “shape” than the other posts 
# there is no "tags" field and we’ve added a new field, "retenion". 
# This is what we mean when we say that MongoDB is schema-free.

new_posts = [{"reinsurer": "AIG",
              "treaty": "XOL Layer 2018",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2018, 11, 12, 11, 14)},
              {"reinsurer": "Munich Re",
               "treaty": "QS 2018",
               "retenion": "QS 20% for US business",
               "date": datetime.datetime(2018, 11, 10, 10, 45)}]

result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('5b747dcf760df74924ecbe2a'), ObjectId('5b747dcf760df74924ecbe2b')]

#### * The result from insert_many() now returns two ObjectId instances, one for each inserted document.*
- new_posts[1] has a different “shape” than the other posts - there is no "tags" field and we’ve added a new field, "title".
- This is what we mean when we say that MongoDB is schema-free.

In [17]:
for post in posts.find():
     pprint.pprint(post)

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b733fb2a68b144b14aa4fdb'),
 'date': datetime.datetime(2018, 8, 14, 20, 46, 42, 222000),
 'reinsurer': 'Swiss Re',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'Clash Layer'}
{'_id': ObjectId('5b7340e5a68b144b14aa4fdc'),
 'date': datetime.datetime(2018, 11, 12, 11, 14),
 'reinsurer': 'AIG',
 'tags': ['bulk', 'insert'],
 'treaty': 'XOL Layer 2018'}
{'_id': ObjectId('5b7340e5a68b144b14aa4fdd'),
 'date': datetime.datetime(2018, 11, 10, 10, 45),
 'reinsurer': 'Munich Re',
 'retenion': 'QS 20% for US business',
 'treaty': 'QS 2018'}
{'_id': ObjectId('5b74590c760df751508616c1'),
 'date': datetime.datetime(2018, 8, 15, 16, 47, 4, 939000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b74594e760df751508616c2'),
 'da

In [20]:
#### **Below is the screenshot of the MongoDB structures showing list of dataset**
<img src="/images/MongoDB1.png" alt="[img: MongoDB view]" title="MongoDB View" />

SyntaxError: invalid syntax (<ipython-input-20-ea8111e655ce>, line 3)

In [21]:
# different than find_one
for post in posts.find({"reinsurer": "AIG"}):
    pprint.pprint(post)

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b7340e5a68b144b14aa4fdc'),
 'date': datetime.datetime(2018, 11, 12, 11, 14),
 'reinsurer': 'AIG',
 'tags': ['bulk', 'insert'],
 'treaty': 'XOL Layer 2018'}
{'_id': ObjectId('5b74590c760df751508616c1'),
 'date': datetime.datetime(2018, 8, 15, 16, 47, 4, 939000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b745967760df751508616c3'),
 'date': datetime.datetime(2018, 11, 12, 11, 14),
 'reinsurer': 'AIG',
 'tags': ['bulk', 'insert'],
 'treaty': 'XOL Layer 2018'}
{'_id': ObjectId('5b747db3760df74924ecbe28'),
 'date': datetime.datetime(2018, 8, 15, 19, 23, 28, 222000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b747dcf760df74924ecbe2a'),
 'date': datetime.da

In [22]:
pprint.pprint(posts.find_one({"reinsurer": "AIG"}))

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}


#### * Counting*
- If we just want to know how many documents match a query we can perform a count() operation instead of a full query.
- We can get a count of all of the documents in a collection:

In [23]:
posts.count()

C:\apps\python36\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


12

In [24]:
posts.find({"reinsurer": "AIG"}).count()

C:\apps\python36\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


6

#### * Range Queries*
- MongoDB supports many different types of advanced queries.
- As an example, lets perform a query where we limit results to posts older than a certain date,
- but also sort the results by author:

In [25]:
d = datetime.datetime(2018, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("reinsurer"):
    pprint.pprint(post)

{'_id': ObjectId('5b733f8da68b144b14aa4fda'),
 'date': datetime.datetime(2018, 8, 14, 20, 45, 55, 156000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b7340e5a68b144b14aa4fdc'),
 'date': datetime.datetime(2018, 11, 12, 11, 14),
 'reinsurer': 'AIG',
 'tags': ['bulk', 'insert'],
 'treaty': 'XOL Layer 2018'}
{'_id': ObjectId('5b74590c760df751508616c1'),
 'date': datetime.datetime(2018, 8, 15, 16, 47, 4, 939000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b745967760df751508616c3'),
 'date': datetime.datetime(2018, 11, 12, 11, 14),
 'reinsurer': 'AIG',
 'tags': ['bulk', 'insert'],
 'treaty': 'XOL Layer 2018'}
{'_id': ObjectId('5b747db3760df74924ecbe28'),
 'date': datetime.datetime(2018, 8, 15, 19, 23, 28, 222000),
 'reinsurer': 'AIG',
 'tags': ['mongodb', 'python', 'pymongo'],
 'treaty': 'XOL layer'}
{'_id': ObjectId('5b747dcf760df74924ecbe2a'),
 'date': datetime.da

#### * Indexing*
- Adding indexes can help accelerate certain queries and can also add additional functionality to querying and storing documents.
- In this example, we’ll demonstrate how to create a unique index on a key that rejects documents whose value for that key already exists in the index.
- First, we’ll need to create the index:

In [26]:
 result = db.profiles.create_index([('user_id', pymongo.ASCENDING)],
                                  unique=True)
 sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

#### *Notice that we have two indexes now*
- one is the index on _id that MongoDB creates automatically,
- and the other is the index on user_id we just created.

In [28]:
 # set up some user profiles:

user_profiles = [
    {'user_id': 218, 'reinsurer': 'Partner Re'},
    {'user_id': 219, 'reinsurer': 'General Re'}]
result = db.profiles.insert_many(user_profiles) 
  

In [32]:
 # DuplicateKeyError: E11000 duplicate key error collection: test_database.profiles index: user_id_1 dup key: { : 212 }
 new_profile = {'user_id': 213, 'reinsurer': 'XL American'}
 duplicate_profile = {'user_id': 212, 'reinsurer': 'SCOR S.E'}
 result = db.profiles.insert_one(new_profile)  # This is fine.
 result = db.profiles.insert_one(duplicate_profile)

#### * Convert Excel to MongoDB*
- https://stackoverflow.com/questions/21901209/convert-excel-to-mongodb
- I want to move data from Excel files into MongoDB.
- The Excel data is updated daily so I want to add the new data on daily reads.
- I'm using Python and so far have only been trying to make a Python dictionary object to represent the Excel data
- just to understand how MongoDB is storing its data but feel that I misunderstand some vital points.

#### *Collection data from twitter into MongoDB*
- ref:  http://pythondata.com/collecting-storing-tweets-with-python-and-mongodb/